<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خط لوله
</font>
</h1>

In [1]:
# import the methods
import pandas as pd
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# display a nice model diagram
set_config(display='diagram')

/home/amin/Desktop/Jump Machine Learning/qenv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
ابتدا نیاز است فایل مجموعه‌داده‌ی بدون مقادیر گم‌شده (<code>beijing_no_missing.csv</code>) را بخوانیم.
</p>

In [2]:
import pandas as pd
# reading the dataset
df = pd.read_csv('beijing_no_missing.csv', encoding='gbk', low_memory=False)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مثال ساده
</font>
</h2>

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
تقسیم مجموعه‌داده
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
ابتدا به عنوان مثال تعدادی از ستون‌های عددی این مجموعه‌داده را انتخاب می‌کنیم.
</p>

In [3]:
target_name = 'price_category'

In [4]:
# select some numerical columns
selected_columns = ['Lng', 'Lat', 'square', 'kitchen', 'ladderRatio']
data = df[selected_columns]
target = df[target_name]

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به کمک تابع <code>train_test_split</code> می‌توانیم مجموعه‌داده را به مجموعه‌های آموزش و اعتبارسنجی تقسیم کنیم (در این مثال، مجموعه‌ی آزمون نداریم).
</p>

In [5]:
# split data to train and validation sets
data_train, data_valid, target_train, target_valid = train_test_split(
    data,
    target,
    random_state=42,
    test_size=0.25
    )

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خط لوله
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون می‌توانیم خط لوله خود را که دارای سه مرحله‌ی زیر است، پیاده کنیم:
</p>

<ol dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
  <li>پر کردن مقادیر گم‌شده با استفاده از میانه</li>
  <li>نرمال‌سازی استاندارد</li>
  <li>اعمال تابع PCA برای کاهش تعداد ابعاد به ۲ </li>
</ol>

In [6]:
# define the pipeline,
pipe = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('std_scaler', StandardScaler()),
        ('dim_reduction', PCA(n_components=2)),
        ]
    )

pipe

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('std_scaler', StandardScaler()),
                ('dim_reduction', PCA(n_components=2))])

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
ابتدا با استفاده از مجموعه‌ی آموزش، پارامترهای مورد نیاز برای خط لوله را تنظیم می‌کنیم:
</p>

In [8]:
# train parameters of imputer, scaler, PCA
pipe.fit(data_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('std_scaler', StandardScaler()),
                ('dim_reduction', PCA(n_components=2))])


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
حال نوبت آن است که از این خط لوله، برای اعمال سلسله‌مراحل و پارامترهای ذخیره شده، بر روی مجموعه‌ی ارزیابی استفاده کنیم:
</p>

In [9]:
# make a prediction
pipe.transform(data_valid)

array([[-2.54414668,  0.92615186],
       [ 1.29579547, -1.2424207 ],
       [-0.35204905, -0.3004523 ],
       ...,
       [-0.21802675,  0.45011495],
       [ 0.03864067, -0.67161941],
       [-0.05561238,  0.41390314]])

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مثال کمی سخت
</font>
</h2>

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
تقسیم مجموعه‌داده
</font>
</h3>

In [10]:
# import extra libraries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
import math

# remove the target label from data
target = df[target_name]
data = df.drop(columns=[target_name])

# select some numerical and categorical columns
numerical_columns = ['Lng', 'Lat', 'followers', 'square', 'kitchen','ladderRatio']
categorical_columns = ['Cid', 'buildingType', 'renovationCondition','buildingStructure','elevator', 'fiveYearsProperty', 'subway', 'district']

# split data to train and validation sets,
data_train, data_valid, target_train, target_valid = train_test_split(
    data[numerical_columns + categorical_columns],
    target,
    random_state=42,
    test_size=0.25
    )

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خط لوله
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
با استفاده از متد <code>ColumnTransformer</code> می‌توانیم خط لوله‌ای طراحی کنیم که هم از مقادیر عددی و هم دسته‌ای پشتیبانی کند.
</p>

In [11]:
# Create the preprocessors,
numeric_preprocessor = Pipeline(
    steps=[
        ('imputation_mean', SimpleImputer(missing_values=math.nan, strategy='mean')),
        ('scaler', StandardScaler())
        ]
    )

categorical_preprocessor = Pipeline(
    steps=[
        ('imputation_constant', SimpleImputer(fill_value=math.nan, strategy='constant')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
        ]
    )
preprocessor = ColumnTransformer(
    transformers = [
        ('categorical_preprocessor', categorical_preprocessor, categorical_columns),
        ('numeric-preprocessor', numeric_preprocessor, numerical_columns),
        ],
    remainder='drop',
    sparse_threshold=0.3,
    n_jobs=-1
    )

In [13]:
# Define the pipeline,
pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('polynomial', PolynomialFeatures(degree=2))
        ]
    )

pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('categorical_preprocessor',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value=nan,
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Cid', 'buildingType',
                                                   'renovationCondition',
                                                   'buildingStructure',
                                                   'elevator',
                                                   'fiveYearsProperty',
                                                   'subway', 'district']),
                                                 ('numeric-preprocessor',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Lng', 'Lat', 'followers',
                                                   'square', 'kitchen',
                                                   'ladderRatio'])])),
                ('polynomial', PolynomialFeatures())])

In [14]:
# set the paramters of the pipeline
pipe.fit(data_train)

/home/amin/Desktop/Jump Machine Learning/qenv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/amin/Desktop/Jump Machine Learning/qenv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('categorical_preprocessor',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value=nan,
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Cid', 'buildingType',
                                                   'renovationCondition',
                                                   'buildingStructure',
                                                   'elevator',
                                                   'fiveYearsProperty',
                                                   'subway', 'district']),
                                                 ('numeric-preprocessor',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Lng', 'Lat', 'followers',
                                                   'square', 'kitchen',
                                                   'ladderRatio'])])),
                ('polynomial', PolynomialFeatures())])

In [15]:
# make a prediction
pipe.transform(data_valid)

/home/amin/Desktop/Jump Machine Learning/qenv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/amin/Desktop/Jump Machine Learning/qenv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


<79705x8010003 sparse matrix of type '<class 'numpy.float64'>'
	with 9563325 stored elements in Compressed Sparse Row format>